In [1]:
import onnxruntime as ort
import numpy as np

# Load the ONNX model
providers = ['CUDAExecutionProvider']

def preprocess_input_numpy(input_images):
    # اطمینان از اینکه تصاویر به float32 تبدیل شده‌اند
    input_images = input_images.astype(np.float32)
    
    # تبدیل محدوده پیکسل‌ها از [0, 255] به [0, 1]
    input_images /= 255.0

    # تبدیل محدوده پیکسل‌ها از [0, 1] به [-1, 1]
    input_images = input_images * 2.0 - 1.0

    return input_images


def DenseNet_feature_extractor_onnx(input_images):
    # پیش‌پردازش ورودی
    input_images = preprocess_input_numpy(input_images)

    # بررسی اینکه ورودی چهار بعدی است
    assert input_images.ndim == 4, "Input must have 4 dimensions (batch_size, height, width, channels)"

    # تهیه دیکشنری ورودی برای ONNX
    inputs = {ort_session.get_inputs()[0].name: input_images}
    
    # انجام پیش‌بینی
    onnx_outputs = ort_session.run(None, inputs)
    
    # استخراج ویژگی‌ها از خروجی‌های ONNX
    feature_extracted_0 = onnx_outputs[0]
    feature_extracted_1 = onnx_outputs[1]

    # محاسبه Global Average Pooling و Flatten برای هر تصویر در مینی‌بتچ
    global_pooling_0 = np.mean(feature_extracted_0, axis=(1, 2))
    global_pooling_1 = np.mean(feature_extracted_1, axis=(1, 2))

    flatten_0 = global_pooling_0.reshape(global_pooling_0.shape[0], -1)
    flatten_1 = global_pooling_1.reshape(global_pooling_1.shape[0], -1)

    # ادغام ویژگی‌ها
    total_feature = np.concatenate([flatten_0, flatten_1], axis=1)
    return total_feature


# Example usage
# input_image = tf.random.uniform((256, 256, 3))  # Replace with actual image data
# features = DenseNet_feature_extractor_onnx(input_image)
# print(features.shape)


# بارگذاری مدل ONNX
onnx_model_path = "/home/hassan-hossein/graph_landmark_face_anti_spoofing/dense_net_model.onnx"
ort_session = ort.InferenceSession(onnx_model_path , providers=providers)

# تابعی برای پیش‌پردازش ورودی مشابه DenseNet اما با استفاده از NumPy

# تابعی برای استفاده از مدل ONNX

# مثال استفاده
input_images = np.random.uniform(0, 255, (10, 256, 256, 3)).astype(np.uint8)  # جایگزین با داده واقعی
features = DenseNet_feature_extractor_onnx(input_images)
print(features.shape)  # باید به شکل (10, feature_length) 

(10, 640)


In [2]:
import onnxruntime as ort

# تنظیمات برای استفاده از GPU
providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']

# ایجاد جلسه استنتاج با استفاده از GPU
ort_session = ort.InferenceSession(onnx_model_path , providers=providers)

print("Available providers:", ort_session.get_providers())


Available providers: ['CUDAExecutionProvider', 'CPUExecutionProvider']


In [3]:
while True :
    features = DenseNet_feature_extractor_onnx(input_images)

KeyboardInterrupt: 

In [6]:
import os.path as osp
import torch.nn as nn
import torch
import numpy as np
from torch.nn import TransformerEncoder, TransformerEncoderLayer
# import os
# from torch_geometric.data import Dataset, download_url
# from torch_geometric.loader import DataLoader
# import pandas as pd
# from performance import performances_val
# from PIL import Image
# from torch.nn import Linear
# import torch.nn.functional as F
# import matplotlib.pyplot as plt
# from torch_geometric.nn import GCNConv
# from torch_geometric.nn import global_mean_pool, global_max_pool
# from torch_geometric.nn.norm import BatchNorm
# from torch_geometric.data import Data
# from sklearn import metrics
# import numpy as np
# from tqdm import tqdm
# from torch_geometric.nn import ChebConv, GraphSAGE,GraphUNet ,TransformerConv
# import cv2



# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
seed = 1378
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
#landmark_colors = np.load("landmark_colors.npy")
#from sklearn.preprocessing import OneHotEncoder
# one_hot_list=[]
# enc = OneHotEncoder()
# enc.fit(np.expand_dims(landmark_colors,axis=1))  
# for x in landmark_colors:
#     one_hot_list.append(enc.transform([[x]]).toarray()[0])
# one_hot_arr  = np.array(one_hot_list)
# one_hot_arr = torch.FloatTensor(one_hot_arr)


In [7]:
device

device(type='cuda')

In [ ]:
import os.path as osp
import GPUtil
import torch.nn as nn
import torch

from torch.nn import TransformerEncoder, TransformerEncoderLayer
import os
from torch_geometric.data import Dataset, download_url
from torch_geometric.loader import DataLoader
import pandas as pd
from performance import performances_val
from PIL import Image
from torch.nn import Linear
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool, global_max_pool
from torch_geometric.nn.norm import BatchNorm
from torch_geometric.data import Data
from sklearn import metrics
import numpy as np
from tqdm import tqdm
from torch_geometric.nn import ChebConv, GraphSAGE,GraphUNet ,TransformerConv
import cv2



# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
seed = 1378
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
landmark_colors = np.load("landmark_colors.npy")
from sklearn.preprocessing import OneHotEncoder
one_hot_list=[]
enc = OneHotEncoder()
enc.fit(np.expand_dims(landmark_colors,axis=1))  
for x in landmark_colors:
    one_hot_list.append(enc.transform([[x]]).toarray()[0])
one_hot_arr  = np.array(one_hot_list)
one_hot_arr = torch.FloatTensor(one_hot_arr)
